In [23]:
from collections import Counter
import math

In [24]:
# genarte candidates
def generate_candidates(normalised_file):
    candidates = set()
    cache = {}
    with open(normalised_file, 'r', encoding='utf-8') as file:
        for line in file:
            sentence = line.strip()
            if not sentence:
                continue
            for word in sentence.split('▁'):
                if not word:
                    continue
                if word in cache:
                    candidates.update(cache[word])
                    continue
                n = len(word)
                word_subs = [word[i:j] 
                            for i in range(n) 
                            for j in range(i+1, n+1)]
                cache[word] = word_subs 
                candidates.update(word_subs)
    return candidates

In [25]:
# frequency and probability functions
def subword_frequency(corpus, subwords):
    freq = Counter()
    with open(corpus, 'r', encoding='utf-8') as file:
        for line in file:
            sentence = line.strip()
            if not sentence:
                continue
            for word in sentence.split('▁'):
                if not word:
                    continue
                for subword in subwords:
                    start = 0
                    while start <= len(word) - len(subword):
                        if word[start:start+len(subword)] == subword:
                            freq[subword] += 1
                            start += len(subword) 
                        else:
                            start += 1
    return freq

def generate_probabilty_distribution(freq):
    total = sum(freq.values())
    prob_dist = {subword: count / total for subword, count in freq.items()}
    return prob_dist

In [26]:
# Clean spaces
def remove_spaces(corpus):
    with open(corpus, 'r', encoding='utf-8') as file:
        text = file.read()       
    text = text.replace(' ', '') 
    text = text.strip()          
    print("Spaces removed")
    print(text)
    return text

In [27]:
# segmnentation and likelihood
def viterbi(corpus, prob_dist, max_word_length=20, prob_unknown=1e-30):
    n = len(corpus)
    dp = [-float('inf')] * (n + 1)
    dp[0] = 0
    backpointer = [None] * (n + 1)
    for i in range(1, n + 1):
        for j in range(max(0, i - max_word_length), i):
            subword = corpus[j:i]
            if subword in prob_dist:
                score = dp[j] + math.log(prob_dist[subword])
            else:
                score = dp[j] + math.log(prob_unknown)
            if score > dp[i]:
                dp[i] = score
                backpointer[i] = j
    segmentation = []
    i = n
    while i > 0:
        j = backpointer[i]
        segmentation.insert(0, corpus[j:i])
        i = j
    return segmentation

def calculate_likelihood(prob_dist, segmentation):
    return sum(math.log(prob_dist.get(subword, 1e-30)) for subword in segmentation)

In [28]:
# EM Training
def em_training(corpus, prob_dist, max_iterations=10, prune_threshold=1e-7):
    candidates = generate_candidates(corpus)
    print(f"Initial candidate vocabulary size: {len(candidates)}")

    freq = subword_frequency(corpus, candidates)
    prob_dist = generate_probabilty_distribution(freq)

    corpus_path = corpus
    corpus = remove_spaces(corpus)
    
    for it in range(max_iterations):
        print(f"\nIteration {it + 1}")
        new_counts = Counter()

        with open(corpus_path, 'r', encoding='utf-8') as file:
            corpus = corpus.replace("▁", "").strip()
            segmentation = viterbi(corpus, prob_dist)
            new_counts.update(segmentation)

        prob_dist = generate_probabilty_distribution(new_counts)

        pruned_candidates = {subword for subword, prob in prob_dist.items() if prob > prune_threshold}
        candidates = pruned_candidates
        prob_dist = {subword: prob for subword, prob in prob_dist.items() if subword in pruned_candidates}
        print(f"Vocabulary size after iteration {it + 1}: {len(candidates)}")
    
    return candidates, prob_dist

In [29]:
# execution
final_candidates, final_prob_dist = em_training(
    'output.txt',
    generate_probabilty_distribution(subword_frequency('output.txt', generate_candidates('output.txt'))),
    prune_threshold=1e-7
)

print(f"Final candidate vocabulary size: {len(final_candidates)}")
print("Final probability distribution size:", final_prob_dist)
print(final_candidates)


Initial candidate vocabulary size: 6511
Spaces removed
it▁is▁a▁truth▁universally▁acknowledged▁,▁that▁a▁single▁man▁in▁posses▁sion▁of▁a▁good▁fortune▁,▁must▁be▁in▁want▁of▁a▁wife▁.▁
however▁little▁known▁the▁feelings▁or▁views▁of▁such▁a▁man▁may▁be▁on▁his▁first▁entering▁a▁neighbourhood▁,▁this▁truth▁is▁so▁well▁fixed▁in▁the▁minds▁of▁the▁surrounding▁families▁,▁that▁he▁is▁considered▁the▁rightful▁property▁of▁some▁one▁or▁other▁of▁their▁daughters▁.▁
“my▁dear▁mr▁.▁
bennet▁,▁”▁said▁his▁lady▁to▁him▁one▁day▁,▁“have▁you▁heard▁that▁netherfield▁park▁is▁let▁at▁last▁?▁”▁mr▁.▁
bennet▁replied▁that▁he▁had▁not▁.▁
“but▁it▁is▁,▁”▁returned▁she▁;▁“for▁mrs▁.▁
long▁has▁just▁been▁here▁,▁and▁she▁told▁me▁all▁about▁it▁.▁”▁mr▁.▁
bennet▁made▁no▁answer▁.▁
“do▁you▁not▁want▁to▁know▁whohas▁taken▁it▁?▁”▁cried▁his▁wife▁impa▁tiently▁.▁
“you▁want▁to▁tell▁me▁,▁and▁i▁have▁no▁objection▁to▁hearing▁it▁.▁”▁this▁was▁invitation▁enough▁.▁
“why▁,▁mydear▁,▁you▁must▁know▁,▁mrs▁.▁
long▁says▁that▁netherfield▁is▁taken▁by▁a▁young▁man▁of▁large▁fort